# (day3) Section 5: Seq2Seq

本書は、「深層学習後編（day3）レポート」の、「Section 5: Seq2Seq」についてのものです。


## 1. 要点まとめ


<!-- - page. 97 - 128

- Section 5: Seq2Seq
  - seq2seq 全体図<br>
    含む "Seq2seq とは"
  - 5-1 Encoder RNN
    - 全体像
    - 処理手順
  - 5-2 Decoder RNN
    - 全体像
    - 処理手順<br>
      含む "Decoder RNN の処理"
    - seq2seq 全体図(具体例)
    - one-hot ベクトル
  - 5-3 HRED
    - seq2seq の課題<br>
      含む "HRED"
    - HRED の構造
    - HRED の課題
  - 5-4 VHRED
  - 5-5 VAE
    - 5-5-1 オートエンコーダ
      - オートエンコーダ構造
      - オートエンコーダ構造図
    - 5-5-2 VAE
      - VAE 構造 -->


### seq2seq 全体図

seq2seq は、 2 つの RNN を結合したもの。主に機械翻訳に使う手法。

page. 98 の図、 1つ目 (左下) の RNN に、文章の単語を順々に入力する。
$ h^{t=T^x} $ に順番にデータが蓄積される。
これが、元の文章の文脈として蓄積される。
図の中心の、「C 文脈」が蓄積された文脈のベクトル表現。


この文脈を、 2 つ目の RNN (右上) に渡す。
別の出力に作り替える。


具体例としては、<br>
1つ目の RNN で日本語で文脈を理解して、<br>
2つ目の RNN で英語で表現する。


1つ目の RNN を、エンコーダ、<br>
2つ目の RNN を、デコーダと言う。

この様なモデルを、 Encoder-Decoder モデルと言う。


Seq2seq は、
Encoder-Decoder モデルの一種。


### 5-1 Encoder RNN


#### 全体像

$$
x^{t=1}, \cdots, x^{t=T^x}
$$

と、文章の単語 ( トークン ) を取り込んで、

$$
x^{t=1} \to h^{t=1}, \cdots, x^{t=T^x} \to h^{t=T^x}
$$

隠れ層に単語の意味が注入されていく。
最終的に、 $ h^{t=T^x} $ に文章全体のベクトル表現が蓄積される。


##### Taking

文章を解析したトークンの一つ一つは、番号付け ( ID ) して、ベクトルの各 1 要素に対応付ける。
例えば、1万語の文章ならば、1万のベクトル要素になる。

この 1 つのベクトルのことを、 one-hot ベクトルと言う。


##### Embedding

one-hot ベクトルは大きいサイズなので、(先の例では、1万要素)<br>
Embedding と言うものを行う。

この Embedding 表現によって、ベクトルのサイズ ( 要素数 ) を数百程度にできる。

Embedding では、機械学習を用いる。
似たような単語では Embedding 表現が似る様に学習を進める。


page. 110 の「演習チャレンジ」の、 `E: word embedding matrix` は、
単語毎に対応する Embedding 表現を格納した行列のこと。

`for w in words` の各単語 'w' ( one-hot vector ) と内積を取ると、その Embedding 表現を取得することができる。

```python
e = E.dot(w)
```


#### 処理手順

1. 繰り返し
  1. 入力 vec_n と前の hidden sate を入力として、hidden state を出力する。

1. 最後の vec_n の入力に対する hidden state が final state.<br>
  final state が入力した分の意味を表すベクトル ( thought vector )


- 事例<br>
  自然言語の分野の最近では、 Google の BART で、以下の様にトークンをベクトルで扱っている。
  - MLM: Masked Language Model<br>
    ランダムに 1 つの単語を削除して、その単語を予測させる様に学習すると、文章全体で、類似の単語のベクトルが学習される。

こういった単語のベクトル ( 特徴ベクトル ) を作成するには、自力で教師なし学習で獲得させることが重要。


### 5-2 Decoder RNN

最初の入力 $ h^{t=1} $ は、 Encoder RNN が作成した、文章の文脈表現。
最終的な出力として、新たな文脈表現を作成する。
( ここでは機械対話の例 )


#### 処理手順

<!-- 1. Encoder RNN の final state ( thought vector ) から、各 token の生成確立を出力する。
  1. final state を Decoder RNN の initial state として設定する。 -->

Encoder RNN の final state ( thought vector ) から、最初の1単語目を作成する。
1単語目ができたら、次々と単語を予測していく。


Decoder のときとは、逆方向に単語を作成する。 ( Detokenize )

$$
embedding \to one-hot \to ID \to word
$$


### 5-3 HRED ( Hierarchical Recurrent Encoder-Decoder )

Seq2seq では、1つの文 ( 文脈表現 ) に対して、1 つの結果しか得られない。
( 機械対話の例では、「一問一答」 )

HRED は、1 つの文だはなく、過去の分も含めた文脈で、結果を得るための仕組み。

- NOTE: 略語: HRED の全文の主典は以下。<br>
  https://arxiv.org/pdf/1507.04808.pdf<br>
  講義動画 5:29:20 で紹介された図は、こちらの page. 3 上部のものである理解。

Seq2seq に Context RNN を追加したもの。
Context RNN は、これまでの会話のコンテキスト ( 文脈 ) 全体を表すベクトルに蓄積。
( context hdden state のことと理解 )
これを次の文の、 seq2seq に引き継ぐことで、会話全体の文脈を維持する。



HRED にも課題がある。文脈の流れと言うより、確率的にありがちな回答を学習してしまう。(会話の中で意味が乏しいものに)


### 5-5 VAE

- NOTE: 講義動画より、 "5-4 VHRED" より先に、こちらを進める。<br>
  ( VRED の前提知識が、 VAE である理解 )


#### 5-5-1 オートエンコーダ

オートエンコーダの構造は、以下の通り。

| 構造 | 説明 | 
| --- | --- |
| 入力 | |
| Encoder | 入力のデータよりも次元削減したデータ z を出力する。<br> Decoder が戻しやすいように学習する。<br>これは効率の良い次元削減したデータの作成を学習することを意味する。 |
| z ( 洗剤変数 ) |  |
| Decoder | z を入力にして、出力が Encoder の入力と一致するように学習する。 |
| 出力 | |

オートエンコーダは、教師なし学習であり、上記の入力である訓練データのみを使用する。


#### 5-5-2 VAE ( Variational AutoEncoder )

VAE の構造としては、オートエンコーダと同様に、以下の通り。
z ( 洗剤変数 ) が異なる。 ( 通常のオートエンコーダでは、どの様な状態か分からない )

| 構造 | 説明 | 
| --- | --- |
| 入力 | |
| Encoder | |
| z ( 洗剤変数 ) | 確率分布 $ z \sim \mathcal{N}(0, 1) $ ( 標準正規分布 ) を仮定したの。<br> L2 ノルムで正規化する。 |
| Decoder | |
| 出力 | |

元のデータの類似度が、潜在変数にも反映させるようにしたいため、
元のデータと同じばらつきになるように、確率分布を適用する。
通常のオートエンコーダより、汎用性を抽出できるようにしたもの。


より汎用的な特徴を得るための学習の工夫として、 Encoder の出力部分と、 Decoder の入力にさらにノイズを加える手法がある。
NN のドロップアウトの様な手法。


### 5-4 VHRED

HRED に VAE を適用したものである。


## 2. 確認テスト

以降の "page. " は、講義資料のページの番号です。


### page. 109

本要点まとめで整理した内容より、 (2) 。

- (1): 双方向 RNN の説明。
- (3): 文全体の表現ベクトルは作成するが、隣接単語からの表現ベクトルではなく、
  Encoder RNN による時間的順序の学習をしている理解。
- (4): GRU


### page. 119

- HRED: seq2seq に、今までの文の文脈 ( Context RNN ) を加えて、会話の文脈で結果を作成できるようにしたもの。

- VHRED: HRED に対して、結果により汎用性を高めるための VAE を適用したもの。


### page. 128

通常のオートエンコーダでは不明な状態である洗剤変数に対して、確率分布を導入したもの。
